In [4]:
# YOLOv5 설치
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17490 (from 3)
Receiving objects: 100% (17496/17496), 16.54 MiB | 18.59 MiB/s, done.
Resolving deltas: 100% (11990/11990), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:0

In [8]:
import torch
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
from models.common import DetectMultiBackend

# 이미지 전처리
img = Image.open("/content/yolov5/Gyeongdan.jpg").convert("RGB")
transform = T.Compose([
    T.Resize((640, 640)),
    T.ToTensor(),
])
input_tensor = transform(img).unsqueeze(0)

# 모델 리스트
model_paths = {
    "food_1_best.pt": "/content/yolov5/food_1_best.pt",
    "food_2_best.pt": "/content/yolov5/food_2_best.pt",
    "food_3_best.pt": "/content/yolov5/food_3_best.pt"
}

# 시각화 + 저장 함수
def visualize_feature_map(feature_map, layer_name, model_label, num_channels=16):
    fmap = feature_map.squeeze(0)
    num_channels = min(num_channels, fmap.shape[0])
    fig, axes = plt.subplots(1, num_channels, figsize=(num_channels * 2, 2))
    for i in range(num_channels):
        axes[i].imshow(fmap[i].cpu(), cmap='viridis')
        axes[i].axis('off')
    fig.suptitle(f"{model_label} - {layer_name}")
    plt.tight_layout()

    # 이미지 저장
    filename = f"{model_label}_{layer_name}.png".replace(" ", "_").replace(".", "")
    filepath = f"/content/yolov5/{filename}"
    plt.savefig(filepath)
    print(f" Saved: {filename}")
    plt.show()

# 디바이스 설정
device = torch.device('cpu')

# 모델마다 중간 레이어 hook 걸고 시각화
for label, path in model_paths.items():
    model = DetectMultiBackend(path, device=device)
    model.model.eval()

    feature_maps = {}
    def hook_fn(module, input, output):
        feature_maps[module] = output

    # 첫 2개 레이어에 hook
    model.model.model[0].register_forward_hook(hook_fn)
    model.model.model[1].register_forward_hook(hook_fn)

    with torch.no_grad():
        _ = model(input_tensor)

    for layer, fmap in feature_maps.items():
        visualize_feature_map(fmap, str(layer.__class__.__name__), label)


Fusing layers... 
Model summary: 212 layers, 20905467 parameters, 0 gradients, 48.0 GFLOPs


✅ Saved: food_1_bestpt_Convpng


Fusing layers... 
Model summary: 157 layers, 7047883 parameters, 0 gradients, 15.9 GFLOPs


✅ Saved: food_1_bestpt_Convpng
✅ Saved: food_2_bestpt_Convpng


Fusing layers... 


✅ Saved: food_2_bestpt_Convpng


Model summary: 157 layers, 7045186 parameters, 0 gradients, 15.9 GFLOPs


✅ Saved: food_3_bestpt_Convpng
✅ Saved: food_3_bestpt_Convpng
